# Loading SAR data with odc-stac

## Imports

In [ ]:
from pystac_client import Client
from odc.stac import load, configure_s3_access
from odc.geo import BoundingBox
import numpy as np
import xarray as xr

## Connect to Digital Earth development STAC catalog

In [ ]:
catalog = "https://explorer.dev.dea.ga.gov.au/stac"

stac_client = Client.open(catalog)

In [ ]:
configure_s3_access(
    cloud_defaults=True,
    aws_unsigned=True,
)

## Define area of interest and date range

In [ ]:
bbox = BoundingBox(
    left=162.994,
    bottom=-77.618,
    right=163.081,
    top=-77.606,
    crs="EPSG:4326"
)

start_date = "2023-05-01"
end_date = "2023-07-01"

## Find STAC items and load into xarray

In [ ]:
collections_query = ["ga_s1_rtc_backscatter_experimental"]
date_query = f"{start_date}/{end_date}"
bbox_query = bbox.bbox

items = stac_client.search(
    collections=collections_query,
    datetime=date_query,
    bbox=bbox_query
).item_collection()

print(f"Found {len(items)} items")

In [ ]:
# Load our filtered data
ds = load(
    items,
    crs="EPSG:3031",
    chunks={},
    resolution=20,
    groupby="solar_day",
    intersects=bbox.boundary(),
).compute()

ds

In [ ]:
ds["HH"].plot.imshow(col="time", col_wrap=3, robust=True, cmap="Greys_r")

## Convert from linear to dB

In [ ]:
ds['HH_db'] = 10 * np.log10(ds["HH"])

ds['HH_db'].plot.imshow(col="time", col_wrap=3, robust=True, cmap="Greys_r")

## Speckle filtering

In [ ]:
# Adapted from https://stackoverflow.com/questions/39785970/speckle-lee-filter-in-python
from scipy.ndimage import uniform_filter

def lee_filter(img, size):
    """
    Applies the Lee filter to reduce speckle noise in an image.

    Parameters:
    img (ndarray): Input image to be filtered.
    size (int): Size of the uniform filter window.

    Returns:
    ndarray: The filtered image.
    """
    img_mean = uniform_filter(img, size)
    img_sqr_mean = uniform_filter(img**2, size)
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = np.var(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output

In [ ]:
# Define a function to apply the Lee filter to a DataArray
def apply_lee_filter(data_array, size=7):
    """
    Applies the Lee filter to the provided DataArray.

    Parameters:
    data_array (xarray.DataArray): The data array to be filtered.
    size (int): Size of the uniform filter window. Default is 7.

    Returns:
    xarray.DataArray: The filtered data array.
    """
    data_array_filled = data_array.fillna(0)
    filtered_data = xr.apply_ufunc(
        lee_filter, data_array_filled,
        kwargs={"size": size},
        input_core_dims=[["y", "x"]],
        output_core_dims=[["y", "x"]],
        dask_gufunc_kwargs={"allow_rechunk": True},
        vectorize=True,
        dask="parallelized",
        output_dtypes=[data_array.dtype]
    )
    return filtered_data

In [ ]:
ds["HH_dB_filtered"] = apply_lee_filter(ds.HH_db, size = 7)

ds['HH_dB_filtered'].plot.imshow(col="time", col_wrap=3, robust=True, cmap="Greys_r")

## View images

In [ ]:
ds['HH_dB_filtered'].isel(time=0).odc.explore(robust=True, cmap="Greys_r")

## Export images

In [ ]:
ds['HH_dB_filtered'].isel(time=0).odc.write_cog("sentinel1_example.tif", overwrite=True)